# Поиск стратегии работы с данными.

В данном ноутбуке рассматривается вопрос о способе работы с данными.
Изначально предполагалось размещать данные но Google Диске в определённой выделенной директории, монитировать диск и работать с ними (данными) напрямую - проводить стат. анализ, использовать для обучения и т.д.. Но как показанно далее такая статегия неприемлема в силу больших временных затрат на обработку данных.

Оценим время доступа к данным посредствам следующих bash утилит из локальной виртуальной среды Google Colab
* file
* imagemagick
* imageinfo

Папку с тренеровочными изображения предворительно размещаем на Google Диске.


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/googleDrive')

Drive already mounted at /googleDrive; to attempt to forcibly remount, call drive.mount("/googleDrive", force_remount=True).


In [ ]:
TRAIN_DATA_PATH = '/googleDrive/MyDrive/ColabNotebooks/ThesisProject/data/train'
test_im_path = '/googleDrive/MyDrive/ColabNotebooks/ThesisProject/data/train/happy/0.jpg'

In [ ]:
! apt-get install file >/dev/null

In [ ]:
! apt-get install imagemagick >/dev/null

In [ ]:
! apt-get install imageinfo >/dev/null # утилита imageinfo, а команда для запуска в bash - identify

In [ ]:
%timeit -n5 -r10 ! file $test_im_path > /dev/null

5 loops, best of 10: 113 ms per loop


In [ ]:
%timeit -n5 -r10 ! identify $test_im_path -format '%w' >/dev/null

5 loops, best of 10: 113 ms per loop


In [ ]:
%timeit -n5 -r10 ! imageinfo --width $test_im_path >/dev/null

5 loops, best of 10: 112 ms per loop


Т.о. все три утилиты показали примерно равное время получения информации о изображении. Давайте оценим время, за которое мы сможем собрать информацию о 50 000 train изображениях:  112ms * 50 000 /1000 /60 /60 = 1.5h. Полтора часа...и это в лучшем случае, т.к. в реальности при циклическом запуске утилиты время затраченное на обработку N изображений больше чем n*N (где n - время затраченное для обработки одного изображения, ~112ms). Поэтому способ обращения с данными при помощи bash утилит нам не подходит. Попробуем использовать модуль PIL (Python Image Library), для этого выполним следующие шаги:

* сформируем список путей до каждого из train изображений
* замерим скорость работы PIL (например получение ширины и высоты изображения) на одном изображении
* замерим скорость работы PIL на нескольких изображений, пробегая по ним в цикле


In [ ]:
import PIL

In [ ]:
# формируем список путей до исследуемых изображений
%%time
im_paths = ! (find $TRAIN_DATA_PATH -type f)

CPU times: user 312 ms, sys: 167 ms, total: 479 ms
Wall time: 3.61 s


In [ ]:
len(im_paths)

50046

In [ ]:
# Замер для одного изображения:
%%time
image = PIL.Image.open(im_paths[0])
width, height = image.size

CPU times: user 923 µs, sys: 51 µs, total: 974 µs
Wall time: 1.9 ms


In [ ]:
# Для 100:
%%time
for path in im_paths[:100]:
    image = PIL.Image.open(path)
    width, height = image.size

CPU times: user 27.1 ms, sys: 13.9 ms, total: 40.9 ms
Wall time: 132 ms


In [ ]:
# Для 500:
%%time
for path in im_paths[:500]:
    image = PIL.Image.open(path)
    width, height = image.size
# 23.7 s

CPU times: user 149 ms, sys: 85.6 ms, total: 235 ms
Wall time: 23.7 s


Как видно из выше проведённых замеров, работа через PIL с небольшим количеством изображений даёт приемленный результат: 100 изображений за 132 сек. Но при обработке уже 500 изображений наблюдаем непропорциональный рост времени обработки. На основе этого факта можно сделать такие выводы:

1. Библиотека PIL быстро обрабатывает изображение (1.9ms на загрузки в память и получение данных о нём).
2. Скорость считывания данных с Google Диска даже при работе в Goole Colab  неравномерная (не одинаковая для разных данных).

Т.о. стратегия с хранение данных на Google Диске и прямым доступом к ним нам не подходит в силу больших временных затрат даже на элементарную аналитику.

Попробуем другую стратегию. Для работы с данными будет загружать их локально на текущую виртуальную машину.



In [ ]:
# Замерим время загрузки архива с данными
%%time
! unzip /googleDrive/MyDrive/ColabNotebooks/ThesisProject/data/train.zip -d /content/ > /dev/null

CPU times: user 96.5 ms, sys: 19 ms, total: 116 ms
Wall time: 36.9 s


In [ ]:
TRAIN_DATA_PATH = '/content/train'

In [ ]:
# Замерим создание списка из путей до каждого файла
%%time
im_paths = ! (find $TRAIN_DATA_PATH -type f )

CPU times: user 256 ms, sys: 161 ms, total: 417 ms
Wall time: 3.21 s


In [ ]:
# Замерим время на получение информации о ширине и высоте каждого изображения
%%time
for path in im_paths :
    image = PIL.Image.open(path)
    width, height = image.size

CPU times: user 3.3 s, sys: 797 ms, total: 4.1 s
Wall time: 4.23 s


Т.о. суммарное время, затраченное на получение информации о размерах изображений (их 50046), не превышает одной минуты. Это приемлемый результат!

Далее в проекте будем именно так работать с данными - хранить архив на Google Диске, а для работы загружать его на рабочую виртуальную машину.